In [1]:
import json
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import ccxt
import requests
import time
import datetime
from datetime import datetime
from sqlalchemy.orm import sessionmaker
from defillama import DefiLlama
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import ccxt
import requests
from datetime import datetime
from defillama import DefiLlama
import json
import sqlite3
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import ccxt
import requests
from datetime import datetime
from defillama import DefiLlama

In [2]:
# Now, let's define the database models using SQLAlchemy:

In [3]:

# Define the database engine
engine = create_engine('sqlite:///crypto_data.db')
def create_database():
    engine = create_engine('sqlite:///crypto_data.db')
    Base.metadata.create_all(engine)
    print("Database created successfully!")

In [77]:
Base = declarative_base()

class Token(Base):
    __tablename__ = 'tokens'
    id = Column(Integer, primary_key=True)
    symbol = Column(String)
    name = Column(String)

class HistoricalMarketCap(Base):
    __tablename__ = 'historical_market_caps'
    id = Column(Integer, primary_key=True)
    token_id = Column(Integer, ForeignKey('tokens.id'))
    source = Column(String)
    timestamp = Column(DateTime)
    market_cap = Column(Float)
    token = relationship('Token', backref='historical_market_caps')
    
    
# Define the ExchangePrice table
class ExchangePrice(Base):
    __tablename__ = 'exchange_prices'
    id = Column(Integer, primary_key=True)
    symbol = Column(String)
    exchange = Column(String)
    timestamp = Column(DateTime)
    open_price = Column(Float)
    high_price = Column(Float)
    low_price = Column(Float)
    close_price = Column(Float)

class DeFiLlamaTVL(Base):
    __tablename__ = 'defillama_tvl'
    id = Column(Integer, primary_key=True)
    protocol = Column(String)
    timestamp = Column(DateTime)
    tvl = Column(Float)


    

In [ ]:
#fetch btc market cap from coingecko

In [78]:
def fetch_historical_data(symbol, name, token_id):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 1000,
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'prices' in data and 'market_caps' in data:
            prices = data['prices']
            market_caps = data['market_caps']
            
            historical_market_caps = []
            for timestamp, cap in market_caps:
                timestamp = datetime.fromtimestamp(timestamp / 1000)
                timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                historical_market_caps.append(
                    {
                        'token_id': token_id,
                        'source': 'Coingecko',
                        'timestamp': timestamp,
                        'market_cap': cap
                    }
                )
            
            return historical_market_caps
        else:
            print(f"Failed to fetch historical data for {symbol} from Coingecko.")
    else:
        print(f"Failed to fetch historical data for {symbol}: {response.status_code}")
    
    return None

def insert_data_into_database(data, name, symbol):
    if data:
        engine = create_engine('sqlite:///crypto_data.db')
        Session = sessionmaker(bind=engine)
        session = Session()

        token = session.query(Token).filter(Token.symbol == symbol).first()
        if token is None:
            token = Token(symbol=symbol, name=name)
            session.add(token)
            session.flush()

        existing_timestamps = set(
            session.query(HistoricalMarketCap.timestamp)
            .filter(HistoricalMarketCap.token_id == token.id)
            .all()
        )
        
        new_data = [
            item for item in data
            if item['timestamp'] not in existing_timestamps
        ]
        
        for item in data:
            timestamp = datetime.strptime(item['timestamp'], '%Y-%m-%d %H:%M:%S')
            historical_market_cap = HistoricalMarketCap(token_id=token.id, source=item['source'], timestamp=timestamp, market_cap=item['market_cap'])
            session.add(historical_market_cap)
        
        session.commit()

        print("Data inserted into the database successfully!")
        print("Preview of fetched data:")
        for item in new_data:
            print(f"Token ID: {item['token_id']}, Source: {item['source']}, Timestamp: {item['timestamp']}, Market Cap: {item['market_cap']}")
    else:
        print("No new data to insert into the database.")

def main():
    token_symbols = [{'symbol': 'bitcoin', 'name': 'Bitcoin', 'token_id': 1}]

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']
        token_id = token['token_id']

        print(f"Fetching data for token with symbol: {symbol}")
        historical_data = fetch_historical_data(symbol, name, token_id)
        insert_data_into_database(historical_data, name, symbol)

if __name__ == '__main__':
    main()

Fetching data for token with symbol: bitcoin
Data inserted into the database successfully!
Preview of fetched data:
Token ID: 1, Source: Coingecko, Timestamp: 2020-09-30 01:00:00, Market Cap: 200532996047.58585
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-01 01:00:00, Market Cap: 199310441688.83572
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-02 01:00:00, Market Cap: 196617081790.5188
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-03 01:00:00, Market Cap: 195660035179.73257
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-04 01:00:00, Market Cap: 195188285650.34625
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-05 01:00:00, Market Cap: 197494664453.7935
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-06 01:00:00, Market Cap: 199530129891.0881
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-07 01:00:00, Market Cap: 196314284797.01337
Token ID: 1, Source: Coingecko, Timestamp: 2020-10-08 01:00:00, Market Cap: 197575523784.584
Token ID: 1, Source: Coingecko, Ti

In [ ]:
#fetch eth market cap from coingecko

In [79]:
def fetch_historical_data(symbol, name, token_id):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 1000,
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'prices' in data and 'market_caps' in data:
            prices = data['prices']
            market_caps = data['market_caps']
            
            historical_market_caps = []
            for timestamp, cap in market_caps:
                timestamp = datetime.fromtimestamp(timestamp / 1000)
                timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                historical_market_caps.append(
                    {
                        'token_id': token_id,
                        'source': 'Coingecko',
                        'timestamp': timestamp,
                        'market_cap': cap
                    }
                )
            
            return historical_market_caps
        else:
            print(f"Failed to fetch historical data for {symbol} from Coingecko.")
    else:
        print(f"Failed to fetch historical data for {symbol}: {response.status_code}")
    
    return None

def insert_data_into_database(data, name, symbol):
    if data:
        engine = create_engine('sqlite:///crypto_data.db')
        Session = sessionmaker(bind=engine)
        session = Session()

        token = session.query(Token).filter(Token.symbol == symbol).first()
        if token is None:
            token = Token(symbol=symbol, name=name)
            session.add(token)
            session.flush()

        existing_timestamps = set(
            session.query(HistoricalMarketCap.timestamp)
            .filter(HistoricalMarketCap.token_id == token.id)
            .all()
        )
        
        new_data = [
            item for item in data
            if item['timestamp'] not in existing_timestamps
        ]
        
        for item in data:
            timestamp = datetime.strptime(item['timestamp'], '%Y-%m-%d %H:%M:%S')
            historical_market_cap = HistoricalMarketCap(token_id=token.id, source=item['source'], timestamp=timestamp, market_cap=item['market_cap'])
            session.add(historical_market_cap)
        
        session.commit()

        print("Data inserted into the database successfully!")
        print("Preview of fetched data:")
        for item in new_data:
            print(f"Token ID: {item['token_id']}, Source: {item['source']}, Timestamp: {item['timestamp']}, Market Cap: {item['market_cap']}")
    else:
        print("No new data to insert into the database.")

def main():
    token_symbols = [{'symbol': 'ethereum', 'name': 'Ethereum', 'token_id': 2}]

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']
        token_id = token['token_id']

        print(f"Fetching data for token with symbol: {symbol}")
        historical_data = fetch_historical_data(symbol, name, token_id)
        insert_data_into_database(historical_data, name, symbol)

if __name__ == '__main__':
    main()

Fetching data for token with symbol: ethereum
Data inserted into the database successfully!
Preview of fetched data:
Token ID: 2, Source: Coingecko, Timestamp: 2020-09-30 01:00:00, Market Cap: 40574256067.82458
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-01 01:00:00, Market Cap: 40586538615.954575
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-02 01:00:00, Market Cap: 39782583382.89109
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-03 01:00:00, Market Cap: 39005631807.605095
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-04 01:00:00, Market Cap: 39098719406.98244
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-05 01:00:00, Market Cap: 39758373727.61445
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-06 01:00:00, Market Cap: 39890007481.293
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-07 01:00:00, Market Cap: 38419834420.75508
Token ID: 2, Source: Coingecko, Timestamp: 2020-10-08 01:00:00, Market Cap: 38556379197.91778
Token ID: 2, Source: Coingecko, Times

In [ ]:
#fetch matic market cap from coingecko

In [80]:
def fetch_historical_data(symbol, name, token_id):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 1000,
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'prices' in data and 'market_caps' in data:
            prices = data['prices']
            market_caps = data['market_caps']
            
            historical_market_caps = []
            for timestamp, cap in market_caps:
                timestamp = datetime.fromtimestamp(timestamp / 1000)
                timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                historical_market_caps.append(
                    {
                        'token_id': token_id,
                        'source': 'Coingecko',
                        'timestamp': timestamp,
                        'market_cap': cap
                    }
                )
            
            return historical_market_caps
        else:
            print(f"Failed to fetch historical data for {symbol} from Coingecko.")
    else:
        print(f"Failed to fetch historical data for {symbol}: {response.status_code}")
    
    return None

def insert_data_into_database(data, name, symbol):
    if data:
        engine = create_engine('sqlite:///crypto_data.db')
        Session = sessionmaker(bind=engine)
        session = Session()

        token = session.query(Token).filter(Token.symbol == symbol).first()
        if token is None:
            token = Token(symbol=symbol, name=name)
            session.add(token)
            session.flush()

        existing_timestamps = set(
            session.query(HistoricalMarketCap.timestamp)
            .filter(HistoricalMarketCap.token_id == token.id)
            .all()
        )
        
        new_data = [
            item for item in data
            if item['timestamp'] not in existing_timestamps
        ]
        
        for item in data:
            timestamp = datetime.strptime(item['timestamp'], '%Y-%m-%d %H:%M:%S')
            historical_market_cap = HistoricalMarketCap(token_id=token.id, source=item['source'], timestamp=timestamp, market_cap=item['market_cap'])
            session.add(historical_market_cap)
        
        session.commit()

        print("Data inserted into the database successfully!")
        print("Preview of fetched data:")
        for item in new_data:
            print(f"Token ID: {item['token_id']}, Source: {item['source']}, Timestamp: {item['timestamp']}, Market Cap: {item['market_cap']}")
    else:
        print("No new data to insert into the database.")

def main():
    token_symbols = [{'symbol': 'matic-network', 'name': 'Matic', 'token_id': 3}]

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']
        token_id = token['token_id']

        print(f"Fetching data for token with symbol: {symbol}")
        historical_data = fetch_historical_data(symbol, name, token_id)
        insert_data_into_database(historical_data, name, symbol)

if __name__ == '__main__':
    main()

Fetching data for token with symbol: matic-network
Data inserted into the database successfully!
Preview of fetched data:
Token ID: 3, Source: Coingecko, Timestamp: 2020-09-30 01:00:00, Market Cap: 74807821.0794714
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-01 01:00:00, Market Cap: 76978925.68164225
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-02 01:00:00, Market Cap: 72254557.32526156
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-03 01:00:00, Market Cap: 69587909.22576936
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-04 01:00:00, Market Cap: 70708954.3640071
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-05 01:00:00, Market Cap: 70823617.10745393
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-06 01:00:00, Market Cap: 71883397.64760992
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-07 01:00:00, Market Cap: 65479097.01041903
Token ID: 3, Source: Coingecko, Timestamp: 2020-10-08 01:00:00, Market Cap: 64626729.43392385
Token ID: 3, Source: Coingecko, Ti

In [ ]:
#fetch litecoin market cap from coingecko

In [81]:
def fetch_historical_data(symbol, name, token_id):
    url = f'https://api.coingecko.com/api/v3/coins/{symbol}/market_chart'
    params = {
        'vs_currency': 'usd',
        'days': 1000,
    }
    
    response = requests.get(url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'prices' in data and 'market_caps' in data:
            prices = data['prices']
            market_caps = data['market_caps']
            
            historical_market_caps = []
            for timestamp, cap in market_caps:
                timestamp = datetime.fromtimestamp(timestamp / 1000)
                timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                historical_market_caps.append(
                    {
                        'token_id': token_id,
                        'source': 'Coingecko',
                        'timestamp': timestamp,
                        'market_cap': cap
                    }
                )
            
            return historical_market_caps
        else:
            print(f"Failed to fetch historical data for {symbol} from Coingecko.")
    else:
        print(f"Failed to fetch historical data for {symbol}: {response.status_code}")
    
    return None

def insert_data_into_database(data, name, symbol):
    if data:
        engine = create_engine('sqlite:///crypto_data.db')
        Session = sessionmaker(bind=engine)
        session = Session()

        token = session.query(Token).filter(Token.symbol == symbol).first()
        if token is None:
            token = Token(symbol=symbol, name=name)
            session.add(token)
            session.flush()

        existing_timestamps = set(
            session.query(HistoricalMarketCap.timestamp)
            .filter(HistoricalMarketCap.token_id == token.id)
            .all()
        )
        
        new_data = [
            item for item in data
            if item['timestamp'] not in existing_timestamps
        ]
        
        for item in data:
            timestamp = datetime.strptime(item['timestamp'], '%Y-%m-%d %H:%M:%S')
            historical_market_cap = HistoricalMarketCap(token_id=token.id, source=item['source'], timestamp=timestamp, market_cap=item['market_cap'])
            session.add(historical_market_cap)
        
        session.commit()

        print("Data inserted into the database successfully!")
        print("Preview of fetched data:")
        for item in new_data:
            print(f"Token ID: {item['token_id']}, Source: {item['source']}, Timestamp: {item['timestamp']}, Market Cap: {item['market_cap']}")
    else:
        print("No new data to insert into the database.")

def main():
    token_symbols = [{'symbol': 'litecoin', 'name': 'litecoin', 'token_id': 4}]

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']
        token_id = token['token_id']

        print(f"Fetching data for token with symbol: {symbol}")
        historical_data = fetch_historical_data(symbol, name, token_id)
        insert_data_into_database(historical_data, name, symbol)

if __name__ == '__main__':
    main()

Fetching data for token with symbol: litecoin
Data inserted into the database successfully!
Preview of fetched data:
Token ID: 4, Source: Coingecko, Timestamp: 2020-09-30 01:00:00, Market Cap: 2992509276.337595
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-01 01:00:00, Market Cap: 3034119907.633546
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-02 01:00:00, Market Cap: 3032516907.158288
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-03 01:00:00, Market Cap: 2944279767.5843797
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-04 01:00:00, Market Cap: 2989137205.153124
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-05 01:00:00, Market Cap: 3061589134.6551595
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-06 01:00:00, Market Cap: 3039737800.0074897
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-07 01:00:00, Market Cap: 3015629548.5997577
Token ID: 4, Source: Coingecko, Timestamp: 2020-10-08 01:00:00, Market Cap: 3055940427.250863
Token ID: 4, Source: Coingecko, T

In [ ]:
#binance price data

In [82]:

# Fetch historical data for a specific symbol and exchange
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.binance.com/api/v3/klines'
    params = {
        'symbol': symbol,
        'interval': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print('Error occurred. Status Code:', response.status_code)
        return None

# Insert data into database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for entry in data:
        timestamp = entry[0]
        readable_timestamp = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(entry[1])
        high_price = float(entry[2])
        low_price = float(entry[3])
        close_price = float(entry[4])

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    token_symbols = [
        {'symbol': 'BTCUSDT', 'name': 'Bitcoin', 'exchange': 'Binance'},
        {'symbol': 'ETHUSDT', 'name': 'Ethereum', 'exchange': 'Binance'},
        {'symbol': 'LTCUSDT', 'name': 'Litecoin ', 'exchange': 'Binance'},
        {'symbol': 'MATICUSDT', 'name': 'Matic ', 'exchange': 'Binance'},
        
        # Add more tokens here
    ]

    interval = '1d'  # 1 day interval
    limit = 30  # Retrieve 30 days of historical data

    for token in token_symbols:
        symbol = token['symbol']
        name = token['name']
        exchange = token['exchange']

        print(f"Fetching data for token with symbol: {symbol} and exchange: {exchange}")
        historical_data = fetch_historical_data(symbol, interval, limit)
        if historical_data:
            insert_data_into_database(historical_data, symbol, exchange)
        else:
            print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database('Binance')

if __name__ == '__main__':
    main()

Fetching data for token with symbol: BTCUSDT and exchange: Binance
Data inserted into the database successfully!
Fetching data for token with symbol: ETHUSDT and exchange: Binance
Data inserted into the database successfully!
Fetching data for token with symbol: LTCUSDT and exchange: Binance
Data inserted into the database successfully!
Fetching data for token with symbol: MATICUSDT and exchange: Binance
Data inserted into the database successfully!
Retrieved data from the database for Binance:
(1, 'BTCUSDT', 'Binance', '2023-05-28 01:00:00', 26854.28, 28261.32, 26764.36, 28065.0)
(2, 'BTCUSDT', 'Binance', '2023-05-29 01:00:00', 28065.01, 28447.14, 27524.6, 27736.4)
(3, 'BTCUSDT', 'Binance', '2023-05-30 01:00:00', 27736.39, 28038.59, 27554.0, 27694.4)
(4, 'BTCUSDT', 'Binance', '2023-05-31 01:00:00', 27694.39, 27835.51, 26839.01, 27210.35)
(5, 'BTCUSDT', 'Binance', '2023-06-01 01:00:00', 27210.36, 27350.0, 26605.05, 26817.93)
(6, 'BTCUSDT', 'Binance', '2023-06-02 01:00:00', 26817.93, 27

In [ ]:
#kraken btc

In [83]:
# Fetch historical data from Kraken
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.kraken.com/0/public/OHLC'
    params = {
        'pair': symbol,
        'interval': interval,
        'since': int(time.time()) - (limit * 86400),  # Calculate the timestamp for the start of the data
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Kraken. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data['result'][symbol]:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[1])
        high_price = float(candle[2])
        low_price = float(candle[3])
        close_price = float(candle[4])

        # Modify the symbol to BTCUSDT
        symbol = 'BTCUSDT'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'XXBTZUSD'  # BTC to USD pair
    interval = '1440'  # 1440 minutes = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Kraken'  # Set the exchange name as 'Kraken'

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, symbol, exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Kraken:
(121, 'BTCUSDT', 'Kraken', '2023-05-28 01:00:00', 26873.7, 28257.0, 26788.7, 28090.0)
(122, 'BTCUSDT', 'Kraken', '2023-05-29 01:00:00', 28090.0, 28450.0, 27550.0, 27741.3)
(123, 'BTCUSDT', 'Kraken', '2023-05-30 01:00:00', 27741.4, 28047.0, 27559.8, 27699.2)
(124, 'BTCUSDT', 'Kraken', '2023-05-31 01:00:00', 27699.2, 27828.6, 26841.3, 27216.2)
(125, 'BTCUSDT', 'Kraken', '2023-06-01 01:00:00', 27215.1, 27341.5, 26571.5, 26814.8)
(126, 'BTCUSDT', 'Kraken', '2023-06-02 01:00:00', 26815.4, 27312.2, 26500.1, 27253.2)
(127, 'BTCUSDT', 'Kraken', '2023-06-03 01:00:00', 27253.1, 27344.6, 26931.8, 27071.9)
(128, 'BTCUSDT', 'Kraken', '2023-06-04 01:00:00', 27072.0, 27489.0, 26967.0, 27122.9)
(129, 'BTCUSDT', 'Kraken', '2023-06-05 01:00:00', 27122.9, 27128.0, 25400.0, 25721.0)
(130, 'BTCUSDT', 'Kraken', '2023-06-06 01:00:00', 25724.8, 27368.4, 25369.0, 27240.1)
(131, 'BTCUSDT', 'Kraken', '2023-06-07 01:00:00',

In [ ]:
#Kraken ETH

In [84]:
# Fetch historical data from Kraken
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.kraken.com/0/public/OHLC'
    params = {
        'pair': symbol,
        'interval': interval,
        'since': int(time.time()) - (limit * 86400),  # Calculate the timestamp for the start of the data
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Kraken. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data['result'][symbol]:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[1])
        high_price = float(candle[2])
        low_price = float(candle[3])
        close_price = float(candle[4])

        # Modify the symbol to BTCUSDT
        symbol = 'ETHUSDT'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'XETHZUSD'  # BTC to USD pair
    interval = '1440'  # 1440 minutes = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Kraken'  # Set the exchange name as 'Kraken'

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, symbol, exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Kraken:
(121, 'BTCUSDT', 'Kraken', '2023-05-28 01:00:00', 26873.7, 28257.0, 26788.7, 28090.0)
(122, 'BTCUSDT', 'Kraken', '2023-05-29 01:00:00', 28090.0, 28450.0, 27550.0, 27741.3)
(123, 'BTCUSDT', 'Kraken', '2023-05-30 01:00:00', 27741.4, 28047.0, 27559.8, 27699.2)
(124, 'BTCUSDT', 'Kraken', '2023-05-31 01:00:00', 27699.2, 27828.6, 26841.3, 27216.2)
(125, 'BTCUSDT', 'Kraken', '2023-06-01 01:00:00', 27215.1, 27341.5, 26571.5, 26814.8)
(126, 'BTCUSDT', 'Kraken', '2023-06-02 01:00:00', 26815.4, 27312.2, 26500.1, 27253.2)
(127, 'BTCUSDT', 'Kraken', '2023-06-03 01:00:00', 27253.1, 27344.6, 26931.8, 27071.9)
(128, 'BTCUSDT', 'Kraken', '2023-06-04 01:00:00', 27072.0, 27489.0, 26967.0, 27122.9)
(129, 'BTCUSDT', 'Kraken', '2023-06-05 01:00:00', 27122.9, 27128.0, 25400.0, 25721.0)
(130, 'BTCUSDT', 'Kraken', '2023-06-06 01:00:00', 25724.8, 27368.4, 25369.0, 27240.1)
(131, 'BTCUSDT', 'Kraken', '2023-06-07 01:00:00',

In [ ]:
#Kraken MATIC

In [85]:
# Fetch historical data from Kraken
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.kraken.com/0/public/OHLC'
    params = {
        'pair': symbol,
        'interval': interval,
        'since': int(time.time()) - (limit * 86400),  # Calculate the timestamp for the start of the data
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Kraken. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data['result'][symbol]:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[1])
        high_price = float(candle[2])
        low_price = float(candle[3])
        close_price = float(candle[4])

        # Modify the symbol to BTCUSDT
        symbol = 'MATICUSDT'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'MATICUSD'  # BTC to USD pair
    interval = '1440'  # 1440 minutes = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Kraken'  # Set the exchange name as 'Kraken'

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, symbol, exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Kraken:
(121, 'BTCUSDT', 'Kraken', '2023-05-28 01:00:00', 26873.7, 28257.0, 26788.7, 28090.0)
(122, 'BTCUSDT', 'Kraken', '2023-05-29 01:00:00', 28090.0, 28450.0, 27550.0, 27741.3)
(123, 'BTCUSDT', 'Kraken', '2023-05-30 01:00:00', 27741.4, 28047.0, 27559.8, 27699.2)
(124, 'BTCUSDT', 'Kraken', '2023-05-31 01:00:00', 27699.2, 27828.6, 26841.3, 27216.2)
(125, 'BTCUSDT', 'Kraken', '2023-06-01 01:00:00', 27215.1, 27341.5, 26571.5, 26814.8)
(126, 'BTCUSDT', 'Kraken', '2023-06-02 01:00:00', 26815.4, 27312.2, 26500.1, 27253.2)
(127, 'BTCUSDT', 'Kraken', '2023-06-03 01:00:00', 27253.1, 27344.6, 26931.8, 27071.9)
(128, 'BTCUSDT', 'Kraken', '2023-06-04 01:00:00', 27072.0, 27489.0, 26967.0, 27122.9)
(129, 'BTCUSDT', 'Kraken', '2023-06-05 01:00:00', 27122.9, 27128.0, 25400.0, 25721.0)
(130, 'BTCUSDT', 'Kraken', '2023-06-06 01:00:00', 25724.8, 27368.4, 25369.0, 27240.1)
(131, 'BTCUSDT', 'Kraken', '2023-06-07 01:00:00',

In [ ]:
#Kraken LTC

In [86]:
# Fetch historical data from Kraken
def fetch_historical_data(symbol, interval, limit):
    base_url = 'https://api.kraken.com/0/public/OHLC'
    params = {
        'pair': symbol,
        'interval': interval,
        'since': int(time.time()) - (limit * 86400),  # Calculate the timestamp for the start of the data
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Kraken. Status code: {response.status_code}")
        return None

# Insert data into database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data['result'][symbol]:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[1])
        high_price = float(candle[2])
        low_price = float(candle[3])
        close_price = float(candle[4])

        # Modify the symbol to BTCUSDT
        symbol = 'LTCUSDT'

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'XLTCZUSD'  # BTC to USD pair
    interval = '1440'  # 1440 minutes = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Kraken'  # Set the exchange name as 'Kraken'

    historical_data = fetch_historical_data(symbol, interval, limit)
    if historical_data:
        insert_data_into_database(historical_data, symbol, exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Kraken:
(121, 'BTCUSDT', 'Kraken', '2023-05-28 01:00:00', 26873.7, 28257.0, 26788.7, 28090.0)
(122, 'BTCUSDT', 'Kraken', '2023-05-29 01:00:00', 28090.0, 28450.0, 27550.0, 27741.3)
(123, 'BTCUSDT', 'Kraken', '2023-05-30 01:00:00', 27741.4, 28047.0, 27559.8, 27699.2)
(124, 'BTCUSDT', 'Kraken', '2023-05-31 01:00:00', 27699.2, 27828.6, 26841.3, 27216.2)
(125, 'BTCUSDT', 'Kraken', '2023-06-01 01:00:00', 27215.1, 27341.5, 26571.5, 26814.8)
(126, 'BTCUSDT', 'Kraken', '2023-06-02 01:00:00', 26815.4, 27312.2, 26500.1, 27253.2)
(127, 'BTCUSDT', 'Kraken', '2023-06-03 01:00:00', 27253.1, 27344.6, 26931.8, 27071.9)
(128, 'BTCUSDT', 'Kraken', '2023-06-04 01:00:00', 27072.0, 27489.0, 26967.0, 27122.9)
(129, 'BTCUSDT', 'Kraken', '2023-06-05 01:00:00', 27122.9, 27128.0, 25400.0, 25721.0)
(130, 'BTCUSDT', 'Kraken', '2023-06-06 01:00:00', 25724.8, 27368.4, 25369.0, 27240.1)
(131, 'BTCUSDT', 'Kraken', '2023-06-07 01:00:00',

In [ ]:
#coinbase BTC

In [87]:
# Fetch historical data for a specific symbol and exchange
def fetch_historical_data(symbol, interval, limit, exchange):
    base_url = 'https://api.pro.coinbase.com/products/{}/candles'.format(symbol)
    params = {
        'granularity': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Coinbase. Status code: {response.status_code}")
        return None

# Insert or update data into the database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[3])
        high_price = float(candle[2])
        low_price = float(candle[1])
        close_price = float(candle[4])

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'BTC-USD'  # Corrected symbol for BTC to USD pair (Coinbase symbol)
    interval = 86400  # 86400 seconds = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Coinbase'  # Specify the exchange name

    historical_data = fetch_historical_data(symbol, interval, limit, exchange)
    if historical_data:
        insert_data_into_database(historical_data, 'BTCUSDT', exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Coinbase:
(241, 'BTCUSDT', 'Coinbase', '2023-06-26 01:00:00', 30480.24, 30662.66, 29900.0, 30135.4)
(242, 'BTCUSDT', 'Coinbase', '2023-06-25 01:00:00', 30547.3, 31057.86, 30288.33, 30480.24)
(243, 'BTCUSDT', 'Coinbase', '2023-06-24 01:00:00', 30709.59, 30815.92, 30265.88, 30547.18)
(244, 'BTCUSDT', 'Coinbase', '2023-06-23 01:00:00', 29886.25, 31443.67, 29802.65, 30707.61)
(245, 'BTCUSDT', 'Coinbase', '2023-06-22 01:00:00', 29999.85, 30513.25, 29539.57, 29886.31)
(246, 'BTCUSDT', 'Coinbase', '2023-06-21 01:00:00', 28320.41, 30800.0, 28271.67, 29995.08)
(247, 'BTCUSDT', 'Coinbase', '2023-06-20 01:00:00', 26839.04, 28417.11, 26637.41, 28320.43)
(248, 'BTCUSDT', 'Coinbase', '2023-06-19 01:00:00', 26336.46, 27040.64, 26250.0, 26837.78)
(249, 'BTCUSDT', 'Coinbase', '2023-06-18 01:00:00', 26507.97, 26693.11, 26245.27, 26336.46)
(250, 'BTCUSDT', 'Coinbase', '2023-06-17 01:00:00', 26329.6, 26783.2, 26165.98, 2650

In [ ]:
#coinbase ETH

In [88]:
# Fetch historical data for a specific symbol and exchange
def fetch_historical_data(symbol, interval, limit, exchange):
    base_url = 'https://api.pro.coinbase.com/products/{}/candles'.format(symbol)
    params = {
        'granularity': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Coinbase. Status code: {response.status_code}")
        return None

# Insert or update data into the database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[3])
        high_price = float(candle[2])
        low_price = float(candle[1])
        close_price = float(candle[4])

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'ETH-USD'  # Corrected symbol for BTC to USD pair (Coinbase symbol)
    interval = 86400  # 86400 seconds = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Coinbase'  # Specify the exchange name

    historical_data = fetch_historical_data(symbol, interval, limit, exchange)
    if historical_data:
        insert_data_into_database(historical_data, 'ETHUSDT', exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Coinbase:
(241, 'BTCUSDT', 'Coinbase', '2023-06-26 01:00:00', 30480.24, 30662.66, 29900.0, 30135.4)
(242, 'BTCUSDT', 'Coinbase', '2023-06-25 01:00:00', 30547.3, 31057.86, 30288.33, 30480.24)
(243, 'BTCUSDT', 'Coinbase', '2023-06-24 01:00:00', 30709.59, 30815.92, 30265.88, 30547.18)
(244, 'BTCUSDT', 'Coinbase', '2023-06-23 01:00:00', 29886.25, 31443.67, 29802.65, 30707.61)
(245, 'BTCUSDT', 'Coinbase', '2023-06-22 01:00:00', 29999.85, 30513.25, 29539.57, 29886.31)
(246, 'BTCUSDT', 'Coinbase', '2023-06-21 01:00:00', 28320.41, 30800.0, 28271.67, 29995.08)
(247, 'BTCUSDT', 'Coinbase', '2023-06-20 01:00:00', 26839.04, 28417.11, 26637.41, 28320.43)
(248, 'BTCUSDT', 'Coinbase', '2023-06-19 01:00:00', 26336.46, 27040.64, 26250.0, 26837.78)
(249, 'BTCUSDT', 'Coinbase', '2023-06-18 01:00:00', 26507.97, 26693.11, 26245.27, 26336.46)
(250, 'BTCUSDT', 'Coinbase', '2023-06-17 01:00:00', 26329.6, 26783.2, 26165.98, 2650

In [ ]:
#coinbase LTC

In [89]:
# Fetch historical data for a specific symbol and exchange
def fetch_historical_data(symbol, interval, limit, exchange):
    base_url = 'https://api.pro.coinbase.com/products/{}/candles'.format(symbol)
    params = {
        'granularity': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Coinbase. Status code: {response.status_code}")
        return None

# Insert or update data into the database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[3])
        high_price = float(candle[2])
        low_price = float(candle[1])
        close_price = float(candle[4])

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'LTC-USD'  # Corrected symbol for BTC to USD pair (Coinbase symbol)
    interval = 86400  # 86400 seconds = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Coinbase'  # Specify the exchange name

    historical_data = fetch_historical_data(symbol, interval, limit, exchange)
    if historical_data:
        insert_data_into_database(historical_data, 'LTCUSDT', exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Coinbase:
(241, 'BTCUSDT', 'Coinbase', '2023-06-26 01:00:00', 30480.24, 30662.66, 29900.0, 30135.4)
(242, 'BTCUSDT', 'Coinbase', '2023-06-25 01:00:00', 30547.3, 31057.86, 30288.33, 30480.24)
(243, 'BTCUSDT', 'Coinbase', '2023-06-24 01:00:00', 30709.59, 30815.92, 30265.88, 30547.18)
(244, 'BTCUSDT', 'Coinbase', '2023-06-23 01:00:00', 29886.25, 31443.67, 29802.65, 30707.61)
(245, 'BTCUSDT', 'Coinbase', '2023-06-22 01:00:00', 29999.85, 30513.25, 29539.57, 29886.31)
(246, 'BTCUSDT', 'Coinbase', '2023-06-21 01:00:00', 28320.41, 30800.0, 28271.67, 29995.08)
(247, 'BTCUSDT', 'Coinbase', '2023-06-20 01:00:00', 26839.04, 28417.11, 26637.41, 28320.43)
(248, 'BTCUSDT', 'Coinbase', '2023-06-19 01:00:00', 26336.46, 27040.64, 26250.0, 26837.78)
(249, 'BTCUSDT', 'Coinbase', '2023-06-18 01:00:00', 26507.97, 26693.11, 26245.27, 26336.46)
(250, 'BTCUSDT', 'Coinbase', '2023-06-17 01:00:00', 26329.6, 26783.2, 26165.98, 2650

In [ ]:
#coinbase MATIC

In [90]:
# Fetch historical data for a specific symbol and exchange
def fetch_historical_data(symbol, interval, limit, exchange):
    base_url = 'https://api.pro.coinbase.com/products/{}/candles'.format(symbol)
    params = {
        'granularity': interval,
        'limit': limit
    }
    response = requests.get(base_url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch historical data from Coinbase. Status code: {response.status_code}")
        return None

# Insert or update data into the database
def insert_data_into_database(data, symbol, exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute('CREATE TABLE IF NOT EXISTS exchange_prices (id INTEGER PRIMARY KEY, symbol TEXT, exchange TEXT, timestamp TEXT, open_price REAL, high_price REAL, low_price REAL, close_price REAL)')

    existing_timestamps = set()

    # Retrieve existing timestamps from the database for the given symbol and exchange
    c.execute("SELECT timestamp FROM exchange_prices WHERE symbol=? AND exchange=?", (symbol, exchange))
    rows = c.fetchall()
    for row in rows:
        existing_timestamps.add(row[0])

    for candle in data:
        timestamp = int(candle[0])
        readable_timestamp = datetime.fromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

        # Check if the timestamp already exists in the database for the given symbol and exchange
        if readable_timestamp in existing_timestamps:
            continue  # Skip inserting duplicate data

        open_price = float(candle[3])
        high_price = float(candle[2])
        low_price = float(candle[1])
        close_price = float(candle[4])

        exchange_price = (None, symbol, exchange, readable_timestamp, open_price, high_price, low_price, close_price)
        c.execute('INSERT INTO exchange_prices VALUES (?, ?, ?, ?, ?, ?, ?, ?)', exchange_price)

    conn.commit()
    print('Data inserted into the database successfully!')

    conn.close()

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    conn = sqlite3.connect('crypto_data.db')
    c = conn.cursor()

    c.execute("SELECT * FROM exchange_prices WHERE exchange=?", (exchange,))
    rows = c.fetchall()

    print(f'Retrieved data from the database for {exchange}:')
    for row in rows:
        print(row)

    conn.close()

def main():
    symbol = 'MATIC-USD'  # Corrected symbol for BTC to USD pair (Coinbase symbol)
    interval = 86400  # 86400 seconds = 1 day
    limit = 30  # Fetch 30 days of historical data
    exchange = 'Coinbase'  # Specify the exchange name

    historical_data = fetch_historical_data(symbol, interval, limit, exchange)
    if historical_data:
        insert_data_into_database(historical_data, 'MATICUSDT', exchange)
    else:
        print('Failed to fetch historical data. No data to insert into the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data inserted into the database successfully!
Retrieved data from the database for Coinbase:
(241, 'BTCUSDT', 'Coinbase', '2023-06-26 01:00:00', 30480.24, 30662.66, 29900.0, 30135.4)
(242, 'BTCUSDT', 'Coinbase', '2023-06-25 01:00:00', 30547.3, 31057.86, 30288.33, 30480.24)
(243, 'BTCUSDT', 'Coinbase', '2023-06-24 01:00:00', 30709.59, 30815.92, 30265.88, 30547.18)
(244, 'BTCUSDT', 'Coinbase', '2023-06-23 01:00:00', 29886.25, 31443.67, 29802.65, 30707.61)
(245, 'BTCUSDT', 'Coinbase', '2023-06-22 01:00:00', 29999.85, 30513.25, 29539.57, 29886.31)
(246, 'BTCUSDT', 'Coinbase', '2023-06-21 01:00:00', 28320.41, 30800.0, 28271.67, 29995.08)
(247, 'BTCUSDT', 'Coinbase', '2023-06-20 01:00:00', 26839.04, 28417.11, 26637.41, 28320.43)
(248, 'BTCUSDT', 'Coinbase', '2023-06-19 01:00:00', 26336.46, 27040.64, 26250.0, 26837.78)
(249, 'BTCUSDT', 'Coinbase', '2023-06-18 01:00:00', 26507.97, 26693.11, 26245.27, 26336.46)
(250, 'BTCUSDT', 'Coinbase', '2023-06-17 01:00:00', 26329.6, 26783.2, 26165.98, 2650

In [ ]:
#bittrex ethusdt

In [91]:
# Fetch data from Bittrex
def fetch_data(symbol):
    url = f"https://api.bittrex.com/v3/markets/{symbol}/candles/DAY_1/recent"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from Bittrex. Status code: {response.status_code}")
        return None

# Save data to the database
def save_data_to_database(data, symbol, exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    for candle in data:
        timestamp = datetime.strptime(candle['startsAt'], "%Y-%m-%dT%H:%M:%SZ")
        open_price = float(candle['open'])
        high_price = float(candle['high'])
        low_price = float(candle['low'])
        close_price = float(candle['close'])
        symbol_in_database = 'ETHUSDT'  # Set the symbol as 'ETHUSDT' for consistency
        
        existing_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                            ExchangePrice.exchange == exchange,
                                                            ExchangePrice.timestamp == timestamp).first()
        
        if existing_data:
            # Update existing data
            existing_data.open_price = open_price
            existing_data.high_price = high_price
            existing_data.low_price = low_price
            existing_data.close_price = close_price
        else:
            # Insert new data
            exchange_price = ExchangePrice(
                symbol=symbol_in_database,
                exchange=exchange,
                timestamp=timestamp,
                open_price=open_price,
                high_price=high_price,
                low_price=low_price,
                close_price=close_price
            )
            session.add(exchange_price)

    session.commit()
    session.close()
    print('Data saved to the database successfully!')

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    symbol_in_database = 'ETHUSDT'
    exchange_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                        ExchangePrice.exchange == exchange).all()

    print(f'{exchange} data from the database:')
    for data in exchange_data:
        print(f"Timestamp: {data.timestamp}, Open: {data.open_price}, High: {data.high_price}, Low: {data.low_price}, Close: {data.close_price}, Exchange: {data.exchange}")

    session.close()

def main():
    symbol = 'eth-usdt'  # ETH to USDT pair (Bittrex symbol)
    exchange = 'Bittrex'  # Set the exchange name as 'Bittrex'

    eth_data = fetch_data(symbol)
    if eth_data:
        save_data_to_database(eth_data, symbol, exchange)
    else:
        print('Failed to fetch data from Bittrex. No data to save to the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data saved to the database successfully!
Bittrex data from the database:
Timestamp: 2022-06-27 00:00:00, Open: 1197.36748436, High: 1237.59591348, Low: 1175.86406859, Close: 1191.99644648, Exchange: Bittrex
Timestamp: 2022-06-28 00:00:00, Open: 1191.97849157, High: 1235.77528253, Low: 1137.36095163, Close: 1144.72710522, Exchange: Bittrex
Timestamp: 2022-06-29 00:00:00, Open: 1144.01375756, High: 1155.61147301, Low: 1088.0, Close: 1098.70472817, Exchange: Bittrex
Timestamp: 2022-06-30 00:00:00, Open: 1098.32910992, High: 1106.27810183, Low: 997.13019422, Close: 1071.05136741, Exchange: Bittrex
Timestamp: 2022-07-01 00:00:00, Open: 1070.52178507, High: 1113.02690297, Low: 1034.40124134, Close: 1056.06601013, Exchange: Bittrex
Timestamp: 2022-07-02 00:00:00, Open: 1059.09462591, High: 1078.05074242, Low: 1028.74826107, Close: 1067.2164386, Exchange: Bittrex
Timestamp: 2022-07-03 00:00:00, Open: 1067.17447247, High: 1087.88490744, Low: 1041.848681, Close: 1074.2304927, Exchange: Bittrex
T

In [ ]:
#btcusdt (bittrex)

In [93]:
# Fetch data from Bittrex
def fetch_data(symbol):
    url = f"https://api.bittrex.com/v3/markets/{symbol}/candles/DAY_1/recent"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from Bittrex. Status code: {response.status_code}")
        return None

# Save data to the database
def save_data_to_database(data, symbol, exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    for candle in data:
        timestamp = datetime.strptime(candle['startsAt'], "%Y-%m-%dT%H:%M:%SZ")
        open_price = float(candle['open'])
        high_price = float(candle['high'])
        low_price = float(candle['low'])
        close_price = float(candle['close'])
        symbol_in_database = 'BTCUSDT'  # Set the symbol as 'ETHUSDT' for consistency
        
        existing_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                            ExchangePrice.exchange == exchange,
                                                            ExchangePrice.timestamp == timestamp).first()
        
        if existing_data:
            # Update existing data
            existing_data.open_price = open_price
            existing_data.high_price = high_price
            existing_data.low_price = low_price
            existing_data.close_price = close_price
        else:
            # Insert new data
            exchange_price = ExchangePrice(
                symbol=symbol_in_database,
                exchange=exchange,
                timestamp=timestamp,
                open_price=open_price,
                high_price=high_price,
                low_price=low_price,
                close_price=close_price
            )
            session.add(exchange_price)

    session.commit()
    session.close()
    print('Data saved to the database successfully!')

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    symbol_in_database = 'BTCUSDT'
    exchange_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                        ExchangePrice.exchange == exchange).all()

    print(f'{exchange} data from the database:')
    for data in exchange_data:
        print(f"Timestamp: {data.timestamp}, Open: {data.open_price}, High: {data.high_price}, Low: {data.low_price}, Close: {data.close_price}, Exchange: {data.exchange}")

    session.close()

def main():
    symbol = 'btc-usdt'  # ETH to USDT pair (Bittrex symbol)
    exchange = 'Bittrex'  # Set the exchange name as 'Bittrex'

    eth_data = fetch_data(symbol)
    if eth_data:
        save_data_to_database(eth_data, symbol, exchange)
    else:
        print('Failed to fetch data from Bittrex. No data to save to the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data saved to the database successfully!
Bittrex data from the database:
Timestamp: 2022-06-27 00:00:00, Open: 21036.4201437, High: 21515.30456337, Low: 20533.70128458, Close: 20733.0, Exchange: Bittrex
Timestamp: 2022-06-28 00:00:00, Open: 20728.65642069, High: 21194.22285834, Low: 20204.75848579, Close: 20276.81526139, Exchange: Bittrex
Timestamp: 2022-06-29 00:00:00, Open: 20282.2716, High: 20418.65412894, Low: 19860.0, Close: 20115.06415473, Exchange: Bittrex
Timestamp: 2022-06-30 00:00:00, Open: 20119.9574, High: 20164.54441355, Low: 18642.45402709, Close: 19939.4373, Exchange: Bittrex
Timestamp: 2022-07-01 00:00:00, Open: 19943.27035434, High: 20870.034, Low: 18992.52397163, Close: 19289.21651422, Exchange: Bittrex
Timestamp: 2022-07-02 00:00:00, Open: 19274.33864246, High: 19437.59263504, Low: 18987.31065903, Close: 19251.7521076, Exchange: Bittrex
Timestamp: 2022-07-03 00:00:00, Open: 19257.77298517, High: 19632.37083322, Low: 18811.0, Close: 19311.24187939, Exchange: Bittrex
T

In [ ]:
#maticusdt (bittrex)

In [94]:
# Fetch data from Bittrex
def fetch_data(symbol):
    url = f"https://api.bittrex.com/v3/markets/{symbol}/candles/DAY_1/recent"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from Bittrex. Status code: {response.status_code}")
        return None

# Save data to the database
def save_data_to_database(data, symbol, exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    for candle in data:
        timestamp = datetime.strptime(candle['startsAt'], "%Y-%m-%dT%H:%M:%SZ")
        open_price = float(candle['open'])
        high_price = float(candle['high'])
        low_price = float(candle['low'])
        close_price = float(candle['close'])
        symbol_in_database = 'MATICUSDT'  # Set the symbol as 'ETHUSDT' for consistency
        
        existing_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                            ExchangePrice.exchange == exchange,
                                                            ExchangePrice.timestamp == timestamp).first()
        
        if existing_data:
            # Update existing data
            existing_data.open_price = open_price
            existing_data.high_price = high_price
            existing_data.low_price = low_price
            existing_data.close_price = close_price
        else:
            # Insert new data
            exchange_price = ExchangePrice(
                symbol=symbol_in_database,
                exchange=exchange,
                timestamp=timestamp,
                open_price=open_price,
                high_price=high_price,
                low_price=low_price,
                close_price=close_price
            )
            session.add(exchange_price)

    session.commit()
    session.close()
    print('Data saved to the database successfully!')

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    symbol_in_database = 'MATICUSDT'
    exchange_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                        ExchangePrice.exchange == exchange).all()

    print(f'{exchange} data from the database:')
    for data in exchange_data:
        print(f"Timestamp: {data.timestamp}, Open: {data.open_price}, High: {data.high_price}, Low: {data.low_price}, Close: {data.close_price}, Exchange: {data.exchange}")

    session.close()

def main():
    symbol = 'matic-usdt'  # ETH to USDT pair (Bittrex symbol)
    exchange = 'Bittrex'  # Set the exchange name as 'Bittrex'

    eth_data = fetch_data(symbol)
    if eth_data:
        save_data_to_database(eth_data, symbol, exchange)
    else:
        print('Failed to fetch data from Bittrex. No data to save to the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data saved to the database successfully!
Bittrex data from the database:
Timestamp: 2022-06-27 00:00:00, Open: 0.56435, High: 0.594, Low: 0.52148, Close: 0.52744, Exchange: Bittrex
Timestamp: 2022-06-28 00:00:00, Open: 0.53167, High: 0.57041, Low: 0.50512, Close: 0.50907, Exchange: Bittrex
Timestamp: 2022-06-29 00:00:00, Open: 0.50958, High: 0.54478, Low: 0.48617, Close: 0.49522, Exchange: Bittrex
Timestamp: 2022-06-30 00:00:00, Open: 0.49477, High: 0.49477, Low: 0.41908, Close: 0.48, Exchange: Bittrex
Timestamp: 2022-07-01 00:00:00, Open: 0.47704, High: 0.4967, Low: 0.45, Close: 0.46249, Exchange: Bittrex
Timestamp: 2022-07-02 00:00:00, Open: 0.46113, High: 0.4936, Low: 0.44935, Close: 0.48984, Exchange: Bittrex
Timestamp: 2022-07-03 00:00:00, Open: 0.49074, High: 0.49386, Low: 0.45065, Close: 0.45824, Exchange: Bittrex
Timestamp: 2022-07-04 00:00:00, Open: 0.45467, High: 0.4943, Low: 0.45197, Close: 0.4943, Exchange: Bittrex
Timestamp: 2022-07-05 00:00:00, Open: 0.48984, High: 0.5336

In [ ]:
#ltcusdt (bittrex)

In [95]:
# Fetch data from Bittrex
def fetch_data(symbol):
    url = f"https://api.bittrex.com/v3/markets/{symbol}/candles/DAY_1/recent"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(f"Failed to fetch data from Bittrex. Status code: {response.status_code}")
        return None

# Save data to the database
def save_data_to_database(data, symbol, exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    for candle in data:
        timestamp = datetime.strptime(candle['startsAt'], "%Y-%m-%dT%H:%M:%SZ")
        open_price = float(candle['open'])
        high_price = float(candle['high'])
        low_price = float(candle['low'])
        close_price = float(candle['close'])
        symbol_in_database = 'MATICUSDT'  # Set the symbol as 'ETHUSDT' for consistency
        
        existing_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                            ExchangePrice.exchange == exchange,
                                                            ExchangePrice.timestamp == timestamp).first()
        
        if existing_data:
            # Update existing data
            existing_data.open_price = open_price
            existing_data.high_price = high_price
            existing_data.low_price = low_price
            existing_data.close_price = close_price
        else:
            # Insert new data
            exchange_price = ExchangePrice(
                symbol=symbol_in_database,
                exchange=exchange,
                timestamp=timestamp,
                open_price=open_price,
                high_price=high_price,
                low_price=low_price,
                close_price=close_price
            )
            session.add(exchange_price)

    session.commit()
    session.close()
    print('Data saved to the database successfully!')

# Retrieve data from the database for a specific exchange
def retrieve_data_from_database(exchange):
    engine = create_engine('sqlite:///crypto_data.db')
    Session = sessionmaker(bind=engine)
    session = Session()

    symbol_in_database = 'MATICUSDT'
    exchange_data = session.query(ExchangePrice).filter(ExchangePrice.symbol == symbol_in_database,
                                                        ExchangePrice.exchange == exchange).all()

    print(f'{exchange} data from the database:')
    for data in exchange_data:
        print(f"Timestamp: {data.timestamp}, Open: {data.open_price}, High: {data.high_price}, Low: {data.low_price}, Close: {data.close_price}, Exchange: {data.exchange}")

    session.close()

def main():
    symbol = 'matic-usdt'  # ETH to USDT pair (Bittrex symbol)
    exchange = 'Bittrex'  # Set the exchange name as 'Bittrex'

    eth_data = fetch_data(symbol)
    if eth_data:
        save_data_to_database(eth_data, symbol, exchange)
    else:
        print('Failed to fetch data from Bittrex. No data to save to the database.')

    retrieve_data_from_database(exchange)

if __name__ == '__main__':
    main()

Data saved to the database successfully!
Bittrex data from the database:
Timestamp: 2022-06-27 00:00:00, Open: 0.56435, High: 0.594, Low: 0.52148, Close: 0.52744, Exchange: Bittrex
Timestamp: 2022-06-28 00:00:00, Open: 0.53167, High: 0.57041, Low: 0.50512, Close: 0.50907, Exchange: Bittrex
Timestamp: 2022-06-29 00:00:00, Open: 0.50958, High: 0.54478, Low: 0.48617, Close: 0.49522, Exchange: Bittrex
Timestamp: 2022-06-30 00:00:00, Open: 0.49477, High: 0.49477, Low: 0.41908, Close: 0.48, Exchange: Bittrex
Timestamp: 2022-07-01 00:00:00, Open: 0.47704, High: 0.4967, Low: 0.45, Close: 0.46249, Exchange: Bittrex
Timestamp: 2022-07-02 00:00:00, Open: 0.46113, High: 0.4936, Low: 0.44935, Close: 0.48984, Exchange: Bittrex
Timestamp: 2022-07-03 00:00:00, Open: 0.49074, High: 0.49386, Low: 0.45065, Close: 0.45824, Exchange: Bittrex
Timestamp: 2022-07-04 00:00:00, Open: 0.45467, High: 0.4943, Low: 0.45197, Close: 0.4943, Exchange: Bittrex
Timestamp: 2022-07-05 00:00:00, Open: 0.48984, High: 0.5336

In [ ]:
# Fetch historical TVLs from DeFi Llama API

In [96]:
def fetch_defillama_data():
    defillama = DefiLlama()
    protocols = defillama.get_all_protocols()

    for protocol in protocols:
        protocol_name = protocol['name']
        protocol_tvls = protocol.get('tvl')

        if protocol_tvls:
            print(f"Protocol: {protocol_name}")
            print("Historical TVLs:")

            if isinstance(protocol_tvls, dict):
                protocol_tvls = [(timestamp, tvl) for timestamp, tvl in protocol_tvls.items()]
            elif isinstance(protocol_tvls, (int, float)):
                protocol_tvls = [(datetime.now(), protocol_tvls)]

            for timestamp, tvl in protocol_tvls:
                if isinstance(timestamp, (int, float)):
                    timestamp = datetime.fromtimestamp(timestamp / 1000).strftime('%Y-%m-%d %H:%M:%S')
                else:
                    timestamp = timestamp.strftime('%Y-%m-%d %H:%M:%S')
                print(f"Timestamp: {timestamp}, TVL: {tvl}")
                
                # Create a connection to the database
                conn = sqlite3.connect('defillama.db')
                c = conn.cursor()
                
                # Execute the SQL statement to insert data
                c.execute("INSERT INTO defillama_tvl (protocol, timestamp, tvl) VALUES (?, ?, ?)",
                          (protocol_name, timestamp, tvl))
                
                # Commit the changes to the database
                conn.commit()
                
                # Close the connection
                conn.close()

    preview_defillama_data_from_database()

def preview_defillama_data_from_database():
    # Create a connection to the database
    conn = sqlite3.connect('defillama.db')
    c = conn.cursor()
    
    # Execute the SQL statement to retrieve data
    c.execute("SELECT * FROM defillama_tvl LIMIT 10")
    
    # Fetch the data and print the preview
    tvl_data = c.fetchall()
    
    for entry in tvl_data:
        print(f"Protocol: {entry[1]}, Timestamp: {entry[2]}, TVL: {entry[3]}")
    
    # Close the connection
    conn.close()

# Call the fetch_defillama_data() function to fetch data from DeFiLlama and save it to the database
fetch_defillama_data()

Protocol: Binance CEX
Historical TVLs:
Timestamp: 2023-06-26 23:09:11, TVL: 58933158423.93671
Protocol: Lido
Historical TVLs:
Timestamp: 2023-06-26 23:09:12, TVL: 13893650482.894308
Protocol: OKX
Historical TVLs:
Timestamp: 2023-06-26 23:09:12, TVL: 10175735284.786278
Protocol: Bitfinex
Historical TVLs:
Timestamp: 2023-06-26 23:09:12, TVL: 9649803633.868383
Protocol: MakerDAO
Historical TVLs:
Timestamp: 2023-06-26 23:09:12, TVL: 6182290895.279748
Protocol: Polygon Bridge & Staking
Historical TVLs:
Timestamp: 2023-06-26 23:09:12, TVL: 5553753157.270024
Protocol: WBTC
Historical TVLs:
Timestamp: 2023-06-26 23:09:13, TVL: 4750741971.985334
Protocol: Curve DEX
Historical TVLs:
Timestamp: 2023-06-26 23:09:13, TVL: 3765457256.863255
Protocol: JustLend
Historical TVLs:
Timestamp: 2023-06-26 23:09:13, TVL: 3738356306.8780327
Protocol: AAVE V2
Historical TVLs:
Timestamp: 2023-06-26 23:09:13, TVL: 3715516753.023892
Protocol: JustCryptos
Historical TVLs:
Timestamp: 2023-06-26 23:09:14, TVL: 35692

Protocol: Acala LCDOT
Historical TVLs:
Timestamp: 2023-06-26 23:09:40, TVL: 122751253.12889229
Protocol: StakeHound
Historical TVLs:
Timestamp: 2023-06-26 23:09:40, TVL: 121241537.52982849
Protocol: Balancer V1
Historical TVLs:
Timestamp: 2023-06-26 23:09:40, TVL: 120165925.63105737
Protocol: cBridge
Historical TVLs:
Timestamp: 2023-06-26 23:09:41, TVL: 117609819.54320522
Protocol: Firi
Historical TVLs:
Timestamp: 2023-06-26 23:09:41, TVL: 116740121.67190938
Protocol: Pendle
Historical TVLs:
Timestamp: 2023-06-26 23:09:41, TVL: 116433326.075504
Protocol: Kava Lend
Historical TVLs:
Timestamp: 2023-06-26 23:09:41, TVL: 115349223.65200883
Protocol: BiSwap
Historical TVLs:
Timestamp: 2023-06-26 23:09:42, TVL: 113667806.21444428
Protocol: BendDAO Lending
Historical TVLs:
Timestamp: 2023-06-26 23:09:42, TVL: 113203831.19888806
Protocol: DefiChain Loans
Historical TVLs:
Timestamp: 2023-06-26 23:09:42, TVL: 107164861.754244
Protocol: Axelar
Historical TVLs:
Timestamp: 2023-06-26 23:09:42, TVL:

Protocol: THENA FUSION
Historical TVLs:
Timestamp: 2023-06-26 23:10:04, TVL: 39684626.52669305
Protocol: Blur Lending
Historical TVLs:
Timestamp: 2023-06-26 23:10:05, TVL: 39552566.68229068
Protocol: ether.fi
Historical TVLs:
Timestamp: 2023-06-26 23:10:05, TVL: 38293448.41401514
Protocol: Inverse Finance FiRM
Historical TVLs:
Timestamp: 2023-06-26 23:10:05, TVL: 36919069.72401875
Protocol: DODO
Historical TVLs:
Timestamp: 2023-06-26 23:10:05, TVL: 36294000.542208835
Protocol: PenPie
Historical TVLs:
Timestamp: 2023-06-26 23:10:05, TVL: 36203940.708571225
Protocol: Flamingo Finance
Historical TVLs:
Timestamp: 2023-06-26 23:10:06, TVL: 35832949.95015561
Protocol: Origin Ether
Historical TVLs:
Timestamp: 2023-06-26 23:10:06, TVL: 35676426.52410189
Protocol: Orca
Historical TVLs:
Timestamp: 2023-06-26 23:10:06, TVL: 35123308.48422176
Protocol: Badger DAO
Historical TVLs:
Timestamp: 2023-06-26 23:10:06, TVL: 34969254.49
Protocol: pNetwork
Historical TVLs:
Timestamp: 2023-06-26 23:10:06, TV

Protocol: Equilibria
Historical TVLs:
Timestamp: 2023-06-26 23:10:40, TVL: 21203636.501686662
Protocol: Toros
Historical TVLs:
Timestamp: 2023-06-26 23:10:40, TVL: 20761809.489016037
Protocol: Liqee
Historical TVLs:
Timestamp: 2023-06-26 23:10:41, TVL: 20743176.264714014
Protocol: Mare Finance
Historical TVLs:
Timestamp: 2023-06-26 23:10:41, TVL: 20667936.74230554
Protocol: Iron Bank
Historical TVLs:
Timestamp: 2023-06-26 23:10:42, TVL: 20488122.15023051
Protocol: Exactly
Historical TVLs:
Timestamp: 2023-06-26 23:10:42, TVL: 20325495.34791465
Protocol: Gravita Protocol
Historical TVLs:
Timestamp: 2023-06-26 23:10:42, TVL: 20305121.978503034
Protocol: Bancor V2.1
Historical TVLs:
Timestamp: 2023-06-26 23:10:43, TVL: 20259105.874750175
Protocol: ioTube
Historical TVLs:
Timestamp: 2023-06-26 23:10:44, TVL: 19911371.226419747
Protocol: Sovryn Dex
Historical TVLs:
Timestamp: 2023-06-26 23:10:45, TVL: 19770227.175751198
Protocol: Tezos Liquidity Baking
Historical TVLs:
Timestamp: 2023-06-26 

Protocol: Swapscanner LSD
Historical TVLs:
Timestamp: 2023-06-26 23:11:09, TVL: 12082095.29098979
Protocol: Opyn Squeeth
Historical TVLs:
Timestamp: 2023-06-26 23:11:09, TVL: 12080052.950383976
Protocol: CLever
Historical TVLs:
Timestamp: 2023-06-26 23:11:10, TVL: 12032971.625099175
Protocol: Pact
Historical TVLs:
Timestamp: 2023-06-26 23:11:10, TVL: 11934479.04616584
Protocol: Rari Capital
Historical TVLs:
Timestamp: 2023-06-26 23:11:10, TVL: 11859094.185757676
Protocol: NodeDAO
Historical TVLs:
Timestamp: 2023-06-26 23:11:10, TVL: 11799771.292472761
Protocol: VyFinance Dex
Historical TVLs:
Timestamp: 2023-06-26 23:11:11, TVL: 11784146.3545268
Protocol: WEMIX.FI AMM
Historical TVLs:
Timestamp: 2023-06-26 23:11:11, TVL: 11782345.59034014
Protocol: Jito
Historical TVLs:
Timestamp: 2023-06-26 23:11:11, TVL: 11732580.696246348
Protocol: Equalizer Exchange
Historical TVLs:
Timestamp: 2023-06-26 23:11:11, TVL: 11585379.90149994
Protocol: Francium
Historical TVLs:
Timestamp: 2023-06-26 23:11

...

Protocol: Kyrios Finance
Historical TVLs:
Timestamp: 2023-06-26 23:26:30, TVL: 3234.597679132856
Protocol: WraithSwap
Historical TVLs:
Timestamp: 2023-06-26 23:26:31, TVL: 3185.9006540078567
Protocol: TFM
Historical TVLs:
Timestamp: 2023-06-26 23:26:31, TVL: 3174.6505523195187
Protocol: Frog Nation Farm
Historical TVLs:
Timestamp: 2023-06-26 23:26:32, TVL: 3138.585007333194
Protocol: VultureSwap
Historical TVLs:
Timestamp: 2023-06-26 23:26:32, TVL: 3136.280514021055
Protocol: Milkydex
Historical TVLs:
Timestamp: 2023-06-26 23:26:32, TVL: 3115.8220480328805
Protocol: 1Beam
Historical TVLs:
Timestamp: 2023-06-26 23:26:33, TVL: 3092.1663540863224
Protocol: BonqDAO
Historical TVLs:
Timestamp: 2023-06-26 23:26:33, TVL: 3066.537615412143
Protocol: Billion Happiness
Historical TVLs:
Timestamp: 2023-06-26 23:26:33, TVL: 3054.8937067296474
Protocol: Bourbon Defi
Historical TVLs:
Timestamp: 2023-06-26 23:26:33, TVL: 3050.8970683682564
Protocol: KungFuu
Historical TVLs:
Timestamp: 2023-06-26 23:2

Protocol: Lavafall
Historical TVLs:
Timestamp: 2023-06-26 23:26:57, TVL: 1287.5081881955487
Protocol: Bring.Finance
Historical TVLs:
Timestamp: 2023-06-26 23:26:57, TVL: 1277.2254648047387
Protocol: Monolith
Historical TVLs:
Timestamp: 2023-06-26 23:26:57, TVL: 1275.3430138330816
Protocol: Cupid Farm
Historical TVLs:
Timestamp: 2023-06-26 23:26:58, TVL: 1255.3116051847114
Protocol: Savanna Finance
Historical TVLs:
Timestamp: 2023-06-26 23:26:58, TVL: 1225.4889935384267
Protocol: XCarnival
Historical TVLs:
Timestamp: 2023-06-26 23:26:58, TVL: 1215.2902083781946
Protocol: BrightSide
Historical TVLs:
Timestamp: 2023-06-26 23:26:58, TVL: 1188.3725011379988
Protocol: OpenBiSea
Historical TVLs:
Timestamp: 2023-06-26 23:26:58, TVL: 1187.6861686708357
Protocol: CoreSwap
Historical TVLs:
Timestamp: 2023-06-26 23:26:59, TVL: 1187.1355198958418
Protocol: AutoBark
Historical TVLs:
Timestamp: 2023-06-26 23:26:59, TVL: 1162.357800014357
Protocol: Zeus Finance
Historical TVLs:
Timestamp: 2023-06-26 2

Protocol: AfraSwap
Historical TVLs:
Timestamp: 2023-06-26 23:27:27, TVL: 387.31264140967755
Protocol: SenpaiSwap
Historical TVLs:
Timestamp: 2023-06-26 23:27:28, TVL: 381.42733229892485
Protocol: JaguarSwap
Historical TVLs:
Timestamp: 2023-06-26 23:27:29, TVL: 377.47627679748985
Protocol: MetaSwap
Historical TVLs:
Timestamp: 2023-06-26 23:27:30, TVL: 374.3281178733943
Protocol: ViveLaBouje
Historical TVLs:
Timestamp: 2023-06-26 23:27:30, TVL: 369.2299963040181
Protocol: ElonSwap
Historical TVLs:
Timestamp: 2023-06-26 23:27:31, TVL: 368.8487189343576
Protocol: Starbank
Historical TVLs:
Timestamp: 2023-06-26 23:27:32, TVL: 364.0249534065016
Protocol: JetMine DAO
Historical TVLs:
Timestamp: 2023-06-26 23:27:32, TVL: 361.84436308307266
Protocol: LazyFi
Historical TVLs:
Timestamp: 2023-06-26 23:27:34, TVL: 357.49341759516335
Protocol: Emerald Swap
Historical TVLs:
Timestamp: 2023-06-26 23:27:37, TVL: 354.6469530573058
Protocol: Evolve
Historical TVLs:
Timestamp: 2023-06-26 23:27:38, TVL: 35

Protocol: DogeClaw
Historical TVLs:
Timestamp: 2023-06-26 23:28:25, TVL: 76.77783192983082
Protocol: WTFDOGE
Historical TVLs:
Timestamp: 2023-06-26 23:28:25, TVL: 75.45978344710296
Protocol: Hotpot Finance
Historical TVLs:
Timestamp: 2023-06-26 23:28:26, TVL: 74.91483135949315
Protocol: Feeder Finance
Historical TVLs:
Timestamp: 2023-06-26 23:28:27, TVL: 66.7327136562892
Protocol: Doge Compounder
Historical TVLs:
Timestamp: 2023-06-26 23:28:28, TVL: 66.6435372621234
Protocol: Silo Protocol
Historical TVLs:
Timestamp: 2023-06-26 23:28:32, TVL: 64.002455297424
Protocol: Ruby Finance
Historical TVLs:
Timestamp: 2023-06-26 23:28:33, TVL: 63.349872146215894
Protocol: Procyon
Historical TVLs:
Timestamp: 2023-06-26 23:28:33, TVL: 60.997004504713395
Protocol: BaksDAO
Historical TVLs:
Timestamp: 2023-06-26 23:28:34, TVL: 60.700462022560814
Protocol: 1Pulse
Historical TVLs:
Timestamp: 2023-06-26 23:28:35, TVL: 60.47932269281162
Protocol: MiniPanther DAO
Historical TVLs:
Timestamp: 2023-06-26 23:

Protocol: Phezzan Protocol
Historical TVLs:
Timestamp: 2023-06-26 23:29:16, TVL: 1.046
Protocol: Infy Protocol
Historical TVLs:
Timestamp: 2023-06-26 23:29:17, TVL: 1.041666
Protocol: Monoceros
Historical TVLs:
Timestamp: 2023-06-26 23:29:17, TVL: 1.00595
Protocol: Kavastarter
Historical TVLs:
Timestamp: 2023-06-26 23:29:17, TVL: 0.9002954999999999
Protocol: Avalps
Historical TVLs:
Timestamp: 2023-06-26 23:29:17, TVL: 0.8380441619626993
Protocol: Welnance
Historical TVLs:
Timestamp: 2023-06-26 23:29:17, TVL: 0.7855642995333088
Protocol: Peaky Finance
Historical TVLs:
Timestamp: 2023-06-26 23:29:18, TVL: 0.7783355705187934
Protocol: BakedPotatoes
Historical TVLs:
Timestamp: 2023-06-26 23:29:18, TVL: 0.7755295913299999
Protocol: Magnet DAO
Historical TVLs:
Timestamp: 2023-06-26 23:29:18, TVL: 0.6640827066067536
Protocol: Maximizer
Historical TVLs:
Timestamp: 2023-06-26 23:29:19, TVL: 0.620701987709394
Protocol: ArbiCheems
Historical TVLs:
Timestamp: 2023-06-26 23:29:19, TVL: 0.6028542419